
# <center>  Tutorial 20 Nov </center>
# <center>  Structure at Macro, Meso and Nano Scale </center>

## <center>  Fabian IJpelaar </center>
### <center>f.ijpelaar@rug.nl </center>
---


# My current project: Building a library for fast simulations of Ising models
- ## We hope that Ising models can provide a bridge between machine learning and material (neuromorphic) computing
___

## Topics:
### - Introduction into Monte Carlo
### - Demonstrations of the Ising Model
### - Ising Models in Machine Learning: The Boltzmann Machine

# Monte Carlo Simulations
- Monte Carlo methods are a class of computational algorithms where we use use random samples to obtain numerical results
- I.e. we might want to calculate averages of some function $\langle f(x) \rangle = \sum_{x} f(x) p(x)$
- Or we might want to generate examples from a probability distirbution

# Monte Carlo Example
## Estimate $\pi$
<center><img src="Circle.png" width="250" align="center"/></center>

### Probability to be in circle: $p_{in} = \frac {A_{Circle}} {A_{Total}} = \frac \pi 4$

In [13]:
include("MakiePi.jl")
estimatePi()

PTimer(InteractiveIsing.var"#490#492"{var"#update#115"{Observable{Int64}, Observable{Int64}, Observable{Vector{Point{2, Float32}}}, Observable{Vector{Point{2, Float32}}}}}(var"#update#115"{Observable{Int64}, Observable{Int64}, Observable{Vector{Point{2, Float32}}}, Observable{Vector{Point{2, Float32}}}}(Observable(0), Observable(2), Observable(Point{2, Float32}[]), Observable(Point{2, Float32}[[0.18154389, 0.272103], [0.39330447, 0.27272093]]))), Timer(Ptr{Nothing} @0x00000002e918a7d0, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x00000002da7433a0, Task (runnable) @0x00000002da7433a0), Base.Threads.SpinLock(0)), true, false), 0.0, 0.016666666666666666)

# Markov Chain Monte Carlo
### - Generating states out of the whole state space with appropriate probability might be hard
### - Solution: Limit generation of states to states that are in the neighborhood, 
### - And define transition probabilities $P(newstate|oldstate)$

### We create a random walk through state space
<center><img src="Markov Chain.png" width="300" align="center"/></center>


# Metropolis Algorithm for Ising Models:
<img src="Metropolis.png" width="400" align="center"/>

### For each spin in the lattice:
1. Randomly select a spin in the lattice.

2. ΔE: Calculate the energy change if that spin would be flipped.

3. If ΔE <= 0 (energy is lowered):
       
    - Flip the spin (accept the new state).

4. Else (ΔE > 0):
       
    - Calculate the acceptance factor, $e^{-\betaΔE}$ i.e. the "Boltzmann Factor"
            
    - r: Generate a random number between 0 and 1.
            
    - If $r < e^{-\betaΔE}$:
        - Flip the spin (accept the new state despite higher energy).
        
    - Else: Do not flip the spin (reject the new state).
5. Go to 1

##### If we choose the acceptance factor to be the Boltzmann factor, over long enough time the distribution will be the same as the Boltzmann Distribution
##### $$ P(\sigma_1,\sigma_2,...) = \frac 1 Z e^{-\beta H} $$



In [1]:
using IJulia
using InteractiveIsing

Starting Interface


Displaying


# Initializing the graph and starting the simulation

In [2]:
# Create the graph
ig = IsingGraph(200,200, type = Discrete)
# 
simulate(ig, gui = false)

# Generate the weights for the IsingModel
wg = @WG "dr -> dr == 1 ? 1 : 0" NN=1
genAdj!(ig[1], wg)

# Create a screen to display the graph
layerwindow = LayerWindow(ig[1]);

UndefVarError: UndefVarError: `LayerWindow` not defined

# Demonstation of the Second Order Phase Transition in the Ising Model

In [20]:
w = createAnalysisWindow(ig, MT_panel, tstep = 0.01);

# Demonstration of Hysteresis
- ### Q: Why do we get hysteresis here?

In [21]:
w = createAnalysisWindow(ig, MB_panel);

# Demonstration of Divergence of Isothermal Susceptibility

- ### We can get the Isothermal Susceptibility from the fluctuations in samples we take of the magnetization
- ### $\chi_T = \left( \frac{\partial M}{\partial H} \right)_T = \beta ( \langle M^2 \rangle - \langle |M| \rangle ^2)$

In [22]:
w = createAnalysisWindow(ig, χₘ_panel, Tχ_panel, shared_interval = 1/500, tstep = 0.01);

# The Anti-Ferromagnetic Ising Model

In [24]:
# If distance is one, connection is -1
wg = @WG "dr -> dr == 1 ? -1 : 0" NN=1 
genAdj!(ig[1], wg)
simulate(ig)

# Boltzmann Machine
## How do Ising models relate to Machine Learning?

### Example: Learning statistics from MNIST
- The MNIST dataset contains images of handrwritten letters of 28x28 pixels

- These define a probability distribution over the pixels $ P_{data}(s_1,s_2,...)$

<img src="MNIST.jpg" width="750" />

### Boltzmann Machine: Take Ising Model and partition into two sets
<img src="BM.png" width="300" />

##### Visible units $v_i$, hidden units $h_i$
##### Probability distribution over visible units $P_v(W_{ij}) = \sum_h \frac 1 Z e^{-BH}$
##### Where $H = -\sum_{ij} W_{ij}\sigma_i\sigma_j -\sum_{i} b_i \sigma_i$

#### Function that gives us the similarity of distributions: KL-Divergence

#### $KL(P_{data}, P_v) = KL(W_{ij}) = \sum_{all states} P_{data} \log \left( \frac{P_{data}}{P_v}\right)$

- #### Gives us a scalar value that decreases when the distributions are similar, increases when they are less similar
- #### $P_{data}$ is fixed, so we can take the derivative w.r.t. the entries in the connection matrix $W_{ij}$
- #### $\frac{\partial KL}{\partial W_{ij}} = -\beta (\langle \sigma_i \sigma_j\rangle_{data} - \langle \sigma_i \sigma_j \rangle_{free}) = -\beta (p_{ij}^{data}-p_{ij}^{free})$


In [8]:
closewindow(layerwindow)

ig = IsingGraph(architecture = [(28,28, Discrete), (32,32, Discrete)], sets = [(0,1),(0,1)])
simulate(ig, overwrite = true)
w2 = LayerWindow(ig[2])


# Visualizing the Connections

In [1]:
include("ImageGrid.jl")
creategrid(conns(ig[1],ig[2]), (28,28), (5,5))

UndefVarError: UndefVarError: `ig` not defined